# Automated ML



Import Dependencies.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-137787
aml-quickstarts-137787
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [3]:
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-137787,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu212"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview


The dataset was download from Kaggle: https://www.kaggle.com/thegurusteam/spanish-high-speed-rail-system-ticket-pricing
Then I have to create some new variables for example day of the week, delete empty rows and create a sample dataset with fewer obs. The reason why I had to perform a sample is that azure needs more than one hour to train an automl model when there are 30.000.000 obs.
Then I uploaded the dataset:


In [15]:
found = False
key = "datalite"
description_text = "dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        print("error")


df = dataset.to_pandas_dataframe()
df.describe()

,duration,departureDay,departureMonth,ALICANTE,BARCELONA,CADIZ,CASTELLO,CASTELLON,CIUDAD REAL,CORDOBA,...,4,5,6,earlyMorning,lateNight,midday,midmorning,morning,night,y
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,2.914604,16.044400,5.584200,0.005800,0.129800,0.00060,0.000800,0.004200,0.001400,0.027400,...,0.144800,0.110400,0.130800,0.007400,0.182400,0.235200,0.177800,0.197200,0.04700,58.024988
std,1.601784,8.725987,2.273575,0.075944,0.336117,0.02449,0.028276,0.064678,0.037394,0.163262,...,0.351934,0.313419,0.337215,0.085713,0.386212,0.424166,0.382383,0.397924,0.21166,25.032202
min,0.380000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,9.450000
25%,1.870000,9.000000,4.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,38.700000
50%,2.530000,16.000000,5.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,55.700000
75%,3.130000,24.000000,7.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,75.400000
max,11.520000,31.000000,12.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,183.500000


## AutoML Configuration



I have choosen regression as task because I need to predic a continue variable. I choose normalized_root_mean_squared_error as primary metric because it is one of the metric that a regression model try to minimize.

In [7]:
#automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}
#automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
# Submit 
pipeline_run = experiment.submit(pipeline)

Created step automl_module [b5d0baed][55bf554c-4179-4587-9ecf-31c519889f89], (This step will run and generate new outputs)
Submitted PipelineRun 8b703b6d-528a-4c96-8a58-009831c94740
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/8b703b6d-528a-4c96-8a58-009831c94740?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-137787/workspaces/quick-starts-ws-137787


## Run Details

In [1]:
# use the `RunDetails` widget to show the different experiments.

**The jupyter notebook didn't save the RunDetails widget plot, you can see it in the screenshots**

In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:

pipeline_run.wait_for_completion()

PipelineRunId: 8b703b6d-528a-4c96-8a58-009831c94740
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/8b703b6d-528a-4c96-8a58-009831c94740?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-137787/workspaces/quick-starts-ws-137787
PipelineRun Status: Running


StepRunId: 9046d2d2-ca62-400a-aadb-a34ee336fe5b
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/9046d2d2-ca62-400a-aadb-a34ee336fe5b?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-137787/workspaces/quick-starts-ws-137787
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '9046d2d2-ca62-400a-aadb-a34ee336fe5b', 'target': 'cpu212', 'status': 'Completed', 'startTimeUtc': '2021-02-07T08:41:54.67698Z', 'endTimeUtc': '2021-02-07T09:10:10.16942Z', 

'Finished'

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [17]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/9046d2d2-ca62-400a-aadb-a34ee336fe5b/metrics_data, 1 files out of an estimated total of 1


In [ ]:
#metrics_output = remote_run.get_pipeline_output(metrics_output_name)
#num_file_downloaded = metrics_output.download('.', show_progress=True)

In [18]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,9046d2d2-ca62-400a-aadb-a34ee336fe5b_28,9046d2d2-ca62-400a-aadb-a34ee336fe5b_18,9046d2d2-ca62-400a-aadb-a34ee336fe5b_30,9046d2d2-ca62-400a-aadb-a34ee336fe5b_11,9046d2d2-ca62-400a-aadb-a34ee336fe5b_23,9046d2d2-ca62-400a-aadb-a34ee336fe5b_4,9046d2d2-ca62-400a-aadb-a34ee336fe5b_10,9046d2d2-ca62-400a-aadb-a34ee336fe5b_14,9046d2d2-ca62-400a-aadb-a34ee336fe5b_17,9046d2d2-ca62-400a-aadb-a34ee336fe5b_26,...,9046d2d2-ca62-400a-aadb-a34ee336fe5b_21,9046d2d2-ca62-400a-aadb-a34ee336fe5b_25,9046d2d2-ca62-400a-aadb-a34ee336fe5b_32,9046d2d2-ca62-400a-aadb-a34ee336fe5b_5,9046d2d2-ca62-400a-aadb-a34ee336fe5b_6,9046d2d2-ca62-400a-aadb-a34ee336fe5b_2,9046d2d2-ca62-400a-aadb-a34ee336fe5b_24,9046d2d2-ca62-400a-aadb-a34ee336fe5b_0,9046d2d2-ca62-400a-aadb-a34ee336fe5b_19,9046d2d2-ca62-400a-aadb-a34ee336fe5b_31
explained_variance,[0.6232936157296693],[0.5036127087234684],[0.4541161233046885],[0.6112223832155453],[0.6266194007464564],[0.6501151966263125],[0.5817335100719511],[0.4801262515500551],[0.6110511007233831],[0.6133802623733096],...,[0.6224203942508165],[0.5160872836925118],[0.6008790762553974],[0.5782741544346126],[0.6171574676266924],[0.6429411676263759],[0.6006569108307119],[0.6966072759901859],[0.5099201161631927],[0.6059313378912327]
spearman_correlation,[0.8047887643435924],[0.7086461878963856],[0.6552561714222768],[0.7982274410137831],[0.8058811176443337],[0.817509172978308],[0.766515412051377],[0.686706005302518],[0.7963765866675135],[0.7931591993933463],...,[0.8010608261669262],[0.7575850193848631],[0.7868757185374514],[0.7681079010608746],[0.7999040442141513],[0.811995550810353],[0.7813991578010775],[0.849848984130305],[0.7180123894530395],[0.7893109022168261]
root_mean_squared_error,[15.35040463034754],[17.631181589502784],[18.495265125878877],[15.592203765514093],[15.287712346796278],[14.804896594211705],[16.17768438689734],[18.03294556238527],[15.601519532983302],[15.559020277678501],...,[15.372678392190295],[17.411872450653078],[15.811189390162932],[16.243639331972425],[15.476326269876255],[14.956522448137028],[15.811873520596869],[13.776467363969862],[17.50160294601295],[15.700534496770906]
normalized_root_mean_squared_log_error,[0.09252003139851082],[0.1095789462997702],[0.11631036471423865],[0.09388794161407986],[0.09305292715620861],[0.09317934511344299],[0.09941034625183527],[0.11314767984170303],[0.09399898345432067],[0.09575899742777626],...,[0.09396677471501408],[0.10995252905839105],[0.09600502124874084],[0.09992688565002984],[0.09352194447266281],[0.094905194045241],[0.09730279351398853],[0.083714151524865],[0.10801903008293574],[0.09580097412113754]
root_mean_squared_log_error,[0.2656294034652699],[0.3146063581905497],[0.33393258010036164],[0.2695567386494806],[0.26715937249021243],[0.2675223244483978],[0.28541182459624587],[0.324852361651],[0.26987554504564154],[0.2749286287378188],...,[0.26978307222565806],[0.31567893203015357],[0.27563497481026644],[0.28689483373637187],[0.2685059434770724],[0.2724773186836469],[0.2793609406085991],[0.24034730420021258],[0.31012776465928765],[0.27504914581772527]
mean_absolute_percentage_error,[21.79269709741438],[27.545627809674073],[30.242110169492832],[22.113676727707258],[22.221687439411784],[22.655034107053293],[24.35145944146438],[29.13122083670443],[22.019004283668647],[23.16681695458531],...,[22.64744092774953],[28.736296421817062],[22.8760493414965],[24.503110703143864],[22.155167271017177],[22.825499528081185],[23.666413787695927],[19.713504084609287],[27.06403425213699],[22.93429851558696]
normalized_mean_absolute_error,[0.0641310348907906],[0.07642775059248093],[0.08246642232851925],[0.06467835655591524],[0.06467166215444514],[0.0643851251847225],[0.07007237501917178],[0.07973475611464446],[0.06472538965232515],[0.0664796960500905],...,[0.06558095965587528],[0.07744187382231564],[0.06656494123182682],[0.06982341578135615],[0.06464490626637016],[0.06480025204509741],[0.06766093262060664],[0.05796798342912409],[0.07587746701400859],[0.06639679433931657]
normalized_median_

In [19]:
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/9046d2d2-ca62-400a-aadb-a34ee336fe5b/model_data, 1 files out of an estimated total of 1


In [20]:
# Save the best model
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingregressor',
  PreFittedSoftVotingRegressor(estimators=[('0',
                                            Pipeline(memory=None,
                                                     steps=[('maxabsscaler',
                                                             MaxAbsScaler(copy=True)),
                                                            ('lightgbmregressor',
                                                             LightGBMRegressor(boosting_type='gbdt',
                                                                               class_weight=None,
                             

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
#from azureml.pipeline.core import PipelineRun
#run_id=""
#pipeline_run=PipelineRun(experiment,run_id)

In [21]:
published_pipeline = pipeline_run.publish_pipeline(
    name="automlModel", description="Training data pipeline", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
automlModel,ed582467-25f2-4b29-8500-e7ce3fa8a66f,Active,REST Endpoint


In [22]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In the cell below, send a request to the web service you deployed to test it.

In [23]:

import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [24]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  4e7ea07e-19f3-4c51-b268-781d5fc506c7


In [27]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In the cell below, print the logs of the web service

In [28]:
published_pipeline_run.wait_for_completion()

PipelineRunId: 56b203f5-9736-4721-9955-b0e9be831db5
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline-rest-endpoint/runs/56b203f5-9736-4721-9955-b0e9be831db5?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-137592/workspaces/quick-starts-ws-137592
PipelineRun Status: Running


StepRunId: 2d2f5c87-e084-4939-9012-47069852ddd5
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline-rest-endpoint/runs/2d2f5c87-e084-4939-9012-47069852ddd5?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-137592/workspaces/quick-starts-ws-137592
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '2d2f5c87-e084-4939-9012-47069852ddd5', 'target': 'cluster212', 'status': 'Completed', 'startTimeUtc': '2021-02-06T17:10:54.983462Z', 'endTimeUtc': '2021-02-06T17:27:43.524972Z', 'properties': {'ContentS

'Finished'